In [28]:
import pandas as pd

all_games_df = pd.read_csv("data/all_games.csv",encoding="utf-8")
all_games_df.drop("Unnamed: 0", axis=1,inplace=True)

In [29]:
from api_helpers.game_stats_helpers import load_past_n_games

columns = ["FG_PCT", "FG3_PCT", "FTM", "OREB", "DREB", "REB", "AST"]
columns_a = [column + "_A" for column in columns]
columns_b = [column + "_B" for column in columns]

n_games_df = load_past_n_games(all_games_df=all_games_df, columns=columns)
n_games_df = n_games_df.dropna()

In [30]:
n_games_df["WL_A"].replace("L", 0, inplace=True)
n_games_df["WL_A"].replace("W", 1, inplace=True)
n_games_df = n_games_df[(n_games_df["WL_A"] == 1) | (n_games_df["WL_A"] == 0)]
n_games_df

/tmp/ipykernel_494135/4085731177.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  n_games_df["WL_A"].replace("L", 0, inplace=True)
/tmp/ipykernel_494135/4085731177.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

,GAME_ID,TEAM_ID_A,TEAM_ID_B,GAME_ID,WL_A,FG_PCT_A,FG3_PCT_A,FTM_A,OREB_A,DREB_A,REB_A,AST_A,FG_PCT_B,FG3_PCT_B,FTM_B,OREB_B,DREB_B,REB_B,AST_B
0,22301159,1610612737,1610612766,22301159,0,0.524000,0.450000,10.000000,7.00,31.000000,38.000000,35.00,0.5060,0.3890,21.0,5.0,28.0,33.0,25.0
1,22301147,1610612737,1610612748,22301147,0,0.524000,0.450000,10.000000,7.00,31.000000,38.000000,35.00,0.4770,0.4000,15.0,5.0,41.0,46.0,29.0
2,22301104,1610612737,1610612765,22301104,1,0.461000,0.333500,10.500000,12.00,36.500000,48.500000,31.50,0.4570,0.3330,17.0,8.0,29.0,37.0,24.0
3,22301076,1610612737,1610612749,22301076,0,0.488667,0.336333,10.333333,10.00,38.333333,48.333333,33.00,0.5510,0.2860,28.0,6.0,34.0,40.0,27.0
4,22301060,1610612737,1610612738,22301060,1,0.480750,0.352250,11.500000,9.75,36.250000,46.000000,30.75,0.4790,0.3890,18.0,9.0,34.0,43.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52802,28800140,1610612766,1610612764,28800140,0,0.473400,0.216600,18.000000,12.80,32.000000,44.800000,26.40,0.4832,0.1000,23.0,15.2,29.4,44.6,24.8
52803,28800115,1610612766,1610612765,28800115,0,0.490600,0.250000,20.400000,11.40,29.200000,40.600000,25.80,0.5160,0.4200,17.8,12.0,29.4,41.4,24.2
52804,28800071,1610612766,1610612751,28800071,0,0.482000,0.200000,17.800000,11.40,29.000000,40.400000,24.40,0.4440,0.2966,23.2,13.4,31.4,44.8,18.8
52805,28800024,1610612766,1610612746,28800024,1,0.476000,0.200000,18.600000,13.40,27.400000,40.800000,24.00,0.4858,0.0800,21.2,13.2,31.8,45.0,22.2


In [31]:
X = n_games_df[columns_a + columns_b]
y = n_games_df["WL_A"]

In [32]:
merged_data = pd.merge(
    all_games_df[columns_a + columns_b], n_games_df, left_index=True, right_index=True
)


columns_x = columns_a + columns_b
columns_x = [column + "_x" for column in columns_x]
X = merged_data[columns_x]
y = n_games_df["WL_A"]

In [33]:
from sklearn.model_selection import train_test_split
import numpy as np


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [34]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import joblib
import sys
from joblib import dump
import pickle
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_test = to_categorical(y_test)
y_train = to_categorical(y_train)
X_train


array([[ 1.23575014, -2.21964063,  0.20045623, ..., -1.98703246,
        -1.32530775,  0.55886635],
       [ 0.03460691, -0.06662262,  1.52445173, ...,  0.32710674,
        -0.57313147,  0.7431149 ],
       [ 0.69438981, -0.60326074,  0.05334562, ..., -1.09697892,
        -1.47574301,  0.19036923],
       ...,
       [-0.99736122,  0.13380849, -0.38798621, ...,  0.32710674,
        -0.1218257 ,  1.29586057],
       [ 0.57596724, -1.13989887,  0.34756684, ..., -1.09697892,
        -0.27226096,  1.11161201],
       [-0.18532072, -0.87481256, -0.09376499, ...,  1.03914957,
         0.9312211 ,  0.55886635]])

In [35]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras

def model_builder(hp):
    model = Sequential()

    hp_units = hp.Int("units", min_value=16, max_value=128, step=32)

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    activations = hp.Choice("activation", values=["relu", "tanh"])
    final_activation = hp.Choice("activation", values=["sigmoid", "softmax"])

    model.add(keras.Input(shape=(14,)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float("dropout", min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp_units, activation=activations))
    model.add(Dropout(hp.Float("dropout", min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp_units, activation=activations))
    model.add(Dropout(hp.Float("dropout", min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(2, activation="sigmoid"))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model




tuner = kt.Hyperband(
    model_builder,
    objective=kt.Objective("val_accuracy", direction="max"),
    max_epochs=10,
    factor=3,
    directory="my_dir",
    project_name="third",
)

print(tuner)

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

tuner.search(X_train, y_train, epochs=5,
             validation_split=0.2, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

print(best_hps)

print(
    f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
"""
)

Trial 30 Complete [00h 00m 10s]
val_accuracy: 0.6017521023750305

Best val_accuracy So Far: 0.6017521023750305
Total elapsed time: 00h 02m 43s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 112 and the optimal learning rate for the optimizer
is 0.0001.

